# Statbotics Python API Documentation

Statbotics.io aims to modernize FRC data analytics through developing and distributing advanced metrics such as Elo and OPR. We have created a Python API to easily query these metrics. This document will serve as a guide to using the API. Enjoy!

The Statbotics API is available on PyPI and can be installed using pip. On your local machine, run **"python -m pip install statbotics"**

Having installed the library, we can now use it in our code. The following cell shows the minimal setup required.

Resources:

* Website: https://statbotics.io/

* Documentation: https://statbotics.readthedocs.io/en/latest/

* Github: https://github.com/avgupta456/statbotics

* Chief Delphi: https://www.chiefdelphi.com/t/introducing-statbotics-io-modernizing-frc-data-analytics/385360

In [1]:
import statbotics

sb = statbotics.Statbotics()

The Statbotics API lets you query the following objects:
1. **Teams**: Get a team's location and Elo statistics (Current Elo, Recent Elo, Mean Elo, Max Elo)
2. **Years**: Get the prediction accuracy and Brier score using Elo and OPR to predict matches, and ILS to predict RPs
3. **TeamYears**: Get a team's performance in a specific year (Start Elo, Pre-Champs Elo, End Elo, ... , Component OPRs, ILS)
4. **Events**: Get summary statistics for a given event (Type, Week, Top 8 Elos, Top 24 Elos, Top 8 OPRs, Top 24 OPRs)
5. **TeamEvents**: Get a team's performance in a specific event (Start Elo, Pre-Playoffs Elo, End Elo, ..., Component OPRs, ILS)
6. **Matches**: Get statistics for a specific match (score breakdown, Elo, OPR, ILS predictions)
7. **TeamMatches**: Get information for a specific team in a specific match (Match, Alliance, Elo)

For each of these seven objects, there are functions to return a specific object, or a list of objects by certain criteria. Each object is returned as a dictionary mapping columns to specific values. Let's look at some examples!

## get_team()

The get_team() function returns information on a specific team. Let's see how we can get the Elo statistics for my team, FRC 5511 Cortechs Robotics.

In [2]:
data  = sb.get_team(5511)
print(data)
print()

print("Team ", data["team"], "has a current Elo of", data["elo"])

{'team': 5511, 'name': 'Cortechs Robotics', 'state': 'NC', 'country': 'USA', 'district': 'fnc', 'active': 1, 'elo': 1699, 'elo_recent': 1657, 'elo_mean': 1603, 'elo_max': 1699, 'wins': 152, 'losses': 117, 'ties': 5, 'count': 274, 'winrate': 0.5639}

Team  5511 has a current Elo of 1699


It's that simple! We can see the name, state, district, country, active status, and elo statistics. As a quick primer, Elo is a measure of team strength, updated after each match based on predicted and actual winning margin. An Elo of 1500 is average and an Elo of 1800 corresponds roughly to the 99% percentile.

## get_teams()

The get_teams() function returns information on multiple teams, and supports multiple filters. We can pass arguments to filter by country, state, district, and active status. We can also order the output by a metric, and limit the output to a number of entries to speed up the query. Check https://statbotics.readthedocs.io/en/latest/ for all the details! For now, let's see how we can get the top 10 teams by recent elo.

In [3]:
print("Top 10 teams by Recent Elo")
print()

data = sb.get_teams(metric="elo_recent", limit=10, fields=["team", "elo_recent"])

print("Rank\t Team\t Elo")
for i, entry in enumerate(data):
    print(i+1, "\t", entry["team"], "\t", entry["elo_recent"])

Top 10 teams by Recent Elo

Rank	 Team	 Elo
1 	 254 	 1977
2 	 2056 	 1950
3 	 1678 	 1915
4 	 148 	 1908
5 	 195 	 1894
6 	 2046 	 1878
7 	 2481 	 1876
8 	 118 	 1874
9 	 2910 	 1863
10 	 225 	 1860


## get_year() and get_years()

These functions return information on year objects. Each dictionary contains the year, accuracy (acc) and mean-squared-error (mse) for match predictions that year (using Elo, OPR, and a mixed approach), and accuracy and mean-squared-error for ranking point predictions (2016-Present) using Caleb Sykes' ILS metric.

In [4]:
print("Years Ranked By Match Prediction Accuracy")
print()

data = sb.get_years(metric="mix_acc", fields=["year", "mix_acc", "mix_mse"])

print("Year\t Accuracy\t MSE")
for entry in data:
    print(entry["year"], "\t", str(entry["mix_acc"]).ljust(6), "\t", entry["mix_mse"])

Years Ranked By Match Prediction Accuracy

Year	 Accuracy	 MSE
2018 	 0.7399 	 0.1744
2013 	 0.7364 	 0.1713
2019 	 0.7133 	 0.1776
2014 	 0.7098 	 0.1857
2015 	 0.7093 	 0.1806
2011 	 0.7075 	 0.1699
2020 	 0.7064 	 0.1858
2016 	 0.7039 	 0.1841
2012 	 0.6939 	 0.1843
2009 	 0.6773 	 0.1971
2006 	 0.6634 	 0.2019
2008 	 0.6598 	 0.193
2017 	 0.6591 	 0.2033
2005 	 0.6542 	 0.2079
2004 	 0.6391 	 0.2046
2010 	 0.6244 	 0.1778
2003 	 0.6215 	 0.2236
2007 	 0.6055 	 0.2139
2002 	 0.5389 	 0.243


## get_team_year() and get_team_years()

We also can get information for a specific team during a specific year (ex. 254 during 2018). The output dictionary includes the team, year, location information, Elo statistics (Start, Pre-Champs, End, Mean, Max, Diff), and component OPR statistics (using ixOPR with 2 iterations). 

Let's see how we can get the top 25 most dominant seasons of all time (keeping in mind it is easier to dominate in certain years).

In [5]:
print("Top 25 End of Season Elos of All Time")
print()

data = sb.get_team_years(metric="elo_max", limit=25, fields=["team", "year", "elo_max"])

print("Rank\t Team\t Year\t Elo Max")
for i, entry in enumerate(data):
    print(i + 1, "\t", entry["team"], "\t", entry["year"], "\t", entry["elo_max"])
print()

Top 25 End of Season Elos of All Time

Rank	 Team	 Year	 Elo Max
1 	 469 	 2010 	 2272
2 	 1114 	 2010 	 2254
3 	 1114 	 2015 	 2215
4 	 254 	 2015 	 2181
5 	 1114 	 2007 	 2155
6 	 1114 	 2013 	 2117
7 	 2056 	 2013 	 2108
8 	 1114 	 2008 	 2105
9 	 469 	 2012 	 2098
10 	 2056 	 2015 	 2086
11 	 1114 	 2014 	 2085
12 	 330 	 2007 	 2077
13 	 254 	 2014 	 2076
14 	 67 	 2010 	 2068
15 	 118 	 2015 	 2067
16 	 254 	 2018 	 2060
17 	 1114 	 2011 	 2059
18 	 1678 	 2015 	 2059
19 	 2056 	 2010 	 2046
20 	 1023 	 2015 	 2041
21 	 2056 	 2018 	 2037
22 	 25 	 2007 	 2036
23 	 25 	 2006 	 2028
24 	 469 	 2013 	 2026
25 	 2056 	 2011 	 2026



## get_event() and get_events()

These two functions return information on events. For a given event, we can get the key, name, week, type (regional, district, etc), Elo statistics (Top 8, Top 24, Mean), and OPR statistics (Top 8, Top 24, Mean).

This example gets the average "Top 8 OPR" by event type (regional, district, district champ, worlds division, einstein)

In [6]:
print("Average Top 8 Elo by Event Type, 2019")
print()

types = {
    0: "regional",
    1: "district",
    2: "district champs",
    3: "worlds division",
    4: "einstein",
}

print("Type\t\t\t Top 8 OPR")
for key, name in types.items():
    data = sb.get_events(year=2019, type=key, fields=["opr_top8"])
    average = round(sum([r["opr_top8"] for r in data if r["opr_top8"] is not None]) / len(data), 2)
    print(name.ljust(15), "\t", average)

Average Top 8 Elo by Event Type, 2019

Type			 Top 8 OPR
regional        	 23.22
district        	 21.52
district champs 	 27.47
worlds division 	 33.2
einstein        	 31.97


## get_team_event() and get_team_events()

Similar to get_team_years(), we can get statistics on how a team fared in a specific event. This information includes the team, event information (week, type, etc), Elo statistics (Start, Pre-Playoffs, End, Mean, Max, Diff), and component OPR statistics.

Let's take a look at 469's start and end Elo per event during the 2010 season, which remains the highest Elo ratings ever recorded in FRC.

In [7]:
print("469 Elo through 2010 Events")
print()

data = sb.get_team_events(team=469, year=2010, metric="-time", fields=["event", "elo_start", "elo_end"])

print("Event Key\t Start\t End Elo")
for entry in data:
    print(entry["event"].ljust(10), "\t", entry["elo_start"], "\t", entry["elo_end"])
print()

469 Elo through 2010 Events

Event Key	 Start	 End Elo
2010dt1    	 1670 	 1772
2010oc     	 1772 	 2172
2010gl     	 2172 	 2229
2010cur    	 2229 	 2190
2010cmp    	 2190 	 2172



## get_match() and get_matches()

If you are interested in score breakdowns and individual predictions, these are the functions for you! These methods return a lot of information on the match (elims, set number, match number), scores (component breakdown for both alliances), and predictions (RP predictions and predicted winner by Elo, OPR, mixed method)

We can see how Statbotics.io predicted matches for 2019 Houston Einstein Championships (Spoiler Alert: 78% Accuracy)

In [8]:
print("Match Predictions for 2019 Houston Champs")
print()

matches = sb.get_matches(event="2019cmptx", elims=True, fields=["key", "winner", "mix_winner", "mix_win_prob"])

print("Match Key\t\t Actual\t\t Predicted\tRed Win Prob")
for match in matches:
    print(match["key"].ljust(15), "\t", match["winner"], "\t\t", match["mix_winner"], "\t\t", match["mix_win_prob"])

Match Predictions for 2019 Houston Champs

Match Key		 Actual		 Predicted	Red Win Prob
2019cmptx_sf1m1 	 blue 		 red 		 0.5327
2019cmptx_sf1m2 	 red 		 red 		 0.5613
2019cmptx_sf1m3 	 blue 		 blue 		 0.1432
2019cmptx_sf1m4 	 blue 		 blue 		 0.3855
2019cmptx_sf1m5 	 red 		 blue 		 0.3037
2019cmptx_sf1m6 	 red 		 red 		 0.8074
2019cmptx_sf1m7 	 blue 		 blue 		 0.3188
2019cmptx_sf1m8 	 blue 		 red 		 0.7062
2019cmptx_sf1m9 	 red 		 blue 		 0.4284
2019cmptx_sf1m10 	 blue 		 blue 		 0.3231
2019cmptx_sf1m11 	 red 		 red 		 0.578
2019cmptx_sf1m12 	 blue 		 red 		 0.5456
2019cmptx_sf1m13 	 blue 		 red 		 0.5963
2019cmptx_sf1m14 	 blue 		 blue 		 0.312
2019cmptx_sf1m15 	 red 		 red 		 0.6359
2019cmptx_f1m1  	 red 		 blue 		 0.3695
2019cmptx_f1m2  	 blue 		 blue 		 0.3838
2019cmptx_f1m3  	 blue 		 blue 		 0.3652


## get_team_match() and get_team_matches()

Our final two methods, these are mostly here for completion, though I'm sure there are creative uses for these. Each dictionary only returns the team, match, alliance, and Elo prior to the match being played. 

In this example, we see how many matches Team 125 plays per year (they've averaged 100+ since 2014!)

In [9]:
print("Team 125 Matches played by Year")
print()

data = sb.get_team_matches(team=125, fields=["year"])

years = {i: 0 for i in range(2002, 2020)}
for match in data:
    years[match["year"]] += 1
    
print("Year\t Matches")
for year, matches in years.items():
    print(year, "\t", matches)
print()

Team 125 Matches played by Year

Year	 Matches
2002 	 18
2003 	 27
2004 	 20
2005 	 8
2006 	 29
2007 	 24
2008 	 26
2009 	 20
2010 	 19
2011 	 34
2012 	 41
2013 	 62
2014 	 98
2015 	 99
2016 	 107
2017 	 107
2018 	 97
2019 	 103



That's all for this demonstration. I hope you have a better understanding of how the API works, and I look forward to seeing all the cool things everyone does with this!